In [34]:
import pytest

import numpy as np
import onnx
import onnx.helper as oh
import onnx.numpy_helper as np_helper
from onnx import TensorProto
from pkgutil import get_data

import finn.core.onnx_exec as oxe
from finn.core.datatype import DataType
from finn.core.modelwrapper import ModelWrapper
from finn.custom_op.general.im2col import compute_conv_output_dim_2D_padding
from finn.custom_op.registry import getCustomOp
from finn.transformation.infer_shapes import InferShapes
from finn.transformation.lower_convs_to_matmul import LowerConvsToMatMul
from finn.util.basic import gen_finn_dt_tensor

In [41]:
from finn.util.visualization import showInNetron
import onnx 

def test_dilation(idt, k_H, k_W, ifm_dim_H, ifm_dim_W, ifm_ch, stride, padding, dilation):

    if k_H > ifm_dim_H:
        pytest.skip("Kernel height must be smaller than image height")
    if k_W > ifm_dim_W:
        pytest.skip("Kernel width must be smaller than image height")

    wdt = idt
    odt = DataType.INT32
    ofm_ch = ifm_ch
    pad_H = padding[0] + padding[2]
    pad_W = padding[1] + padding[3]
    ofm_dim_H = compute_conv_output_dim_2D_padding(ifm_dim_H, k_H, stride, pad_H)
    ofm_dim_W = compute_conv_output_dim_2D_padding(ifm_dim_W, k_W, stride, pad_W)

    # set up onnx model
    inp = oh.make_tensor_value_info(
        "inp", TensorProto.FLOAT, [1, ifm_ch, ifm_dim_H, ifm_dim_W]
    )
    outp = oh.make_tensor_value_info(
        "outp", TensorProto.FLOAT, [1, ofm_ch, ofm_dim_H, ofm_dim_W]
    )

    W = oh.make_tensor_value_info("W", TensorProto.FLOAT, [ofm_ch, ifm_ch, k_H, k_W])

    dw_cnv = oh.make_node(
        "Conv",
        inputs=["inp", "W"],
        outputs=["outp"],
        kernel_shape=[k_H, k_W],
        pads=padding,
        strides=[stride, stride],
        group=ifm_ch,
        dilations=dilation
    )
    graph = oh.make_graph(
        nodes=[dw_cnv],
        name="dw_cnv_graph",
        inputs=[inp],
        outputs=[outp],
        value_info=[W],
    )

    model = oh.make_model(graph, producer_name="dws_cnv-model")
    model = ModelWrapper(model)
    model.set_tensor_datatype("inp", idt)
    model.set_tensor_datatype("outp", odt)
    model.set_tensor_datatype("W", wdt)
    w_tensor = gen_finn_dt_tensor(wdt, [ofm_ch, ifm_ch, k_H, k_W])
    model.set_initializer("W", w_tensor)
        
    model = model.transform(InferShapes())

    ###
    model.save("/tmp/conv_node_test.onnx")
    showInNetron("/tmp/conv_node_test.onnx")
    
    input_tensor = gen_finn_dt_tensor(idt, [1, ifm_ch, ifm_dim_H, ifm_dim_W])
    input_dict = {"inp": input_tensor}
    output_dict = oxe.execute_onnx(model, input_dict)
    expected = output_dict["outp"]

    print(expected)
    
    #model = model.transform(LowerConvsToMatMul())
    #output_dict = oxe.execute_onnx(model, input_dict)
    #produced = output_dict["outp"]
    #assert (produced == expected).all()

    # check if created nodes have attributes that indicate depthwise conv
    #assert model.get_tensor_sparsity("W") is not None
    #im2col_node = getCustomOp(model.graph.node[1])
    #assert im2col_node.get_nodeattr("depthwise") == 1

In [43]:
idt = DataType.INT2
k_H = 2
k_W = 2
ifm_dim_H = 5
ifm_dim_W = 5
ifm_ch = 1
stride = 1
padding = [0,0,0,0]
dilation = [2, 2]

test_dilation(idt, k_H, k_W, ifm_dim_H, ifm_dim_W, ifm_ch, stride, padding, dilation)

RuntimeError: Inferred shape and existing shape differ in dimension 2: (3) vs (4)

In [7]:
def test_conv_lowering_convmnist():

    # load the onnx model
    raw_m = get_data("finn.data", "onnx/mnist-conv/model.onnx")
    model = ModelWrapper(raw_m)
    # model = model.transform(InferShapes())
    # model = model.transform(FoldConstants())
    raw_i = get_data("finn.data", "onnx/mnist-conv/test_data_set_0/input_0.pb")
    input_tensor = onnx.load_tensor_from_string(raw_i)
    input_tensor = np_helper.to_array(input_tensor)
    # execute imported model to get expected answer
    input_name = model.graph.input[0].name
    output_name = model.graph.output[0].name
    input_dict = {input_name: input_tensor}
    output_dict_e = oxe.execute_onnx(model, input_dict)
    expected = output_dict_e[output_name]
    # execute transformed model and compare
    model = model.transform(LowerConvsToMatMul())
    model = model.transform(InferShapes())
    output_dict_p = oxe.execute_onnx(model, input_dict)
    produced = output_dict_p[output_name]
    assert np.isclose(produced, expected).all()

In [8]:
test_conv_lowering_convmnist()

28:28 	 28:28
14:14 	 14:14
28:28 	 28:28
14:14 	 14:14
28:28 	 28:28
14:14 	 14:14


In [21]:
from finn.util.visualization import showInNetron
import onnx 

def test_depthwise_conv_lowering(idt, k_H, k_W, ifm_dim_H, ifm_dim_W, ifm_ch, stride, padding):

    if k_H > ifm_dim_H:
        pytest.skip("Kernel height must be smaller than image height")
    if k_W > ifm_dim_W:
        pytest.skip("Kernel width must be smaller than image height")

    wdt = idt
    odt = DataType.INT32
    ofm_ch = ifm_ch
    pad_H = padding[0] + padding[2]
    pad_W = padding[1] + padding[3]
    ofm_dim_H = compute_conv_output_dim_2D_padding(ifm_dim_H, k_H, stride, pad_H)
    ofm_dim_W = compute_conv_output_dim_2D_padding(ifm_dim_W, k_W, stride, pad_W)

    # set up onnx model
    inp = oh.make_tensor_value_info(
        "inp", TensorProto.FLOAT, [1, ifm_ch, ifm_dim_H, ifm_dim_W]
    )
    outp = oh.make_tensor_value_info(
        "outp", TensorProto.FLOAT, [1, ofm_ch, ofm_dim_H, ofm_dim_W]
    )

    W = oh.make_tensor_value_info("W", TensorProto.FLOAT, [ofm_ch, 1, k_H, k_W])

    dw_cnv = oh.make_node(
        "Conv",
        inputs=["inp", "W"],
        outputs=["outp"],
        kernel_shape=[k_H, k_W],
        pads=padding,
        strides=[stride, stride],
        group=ifm_ch,
    )
    graph = oh.make_graph(
        nodes=[dw_cnv],
        name="dw_cnv_graph",
        inputs=[inp],
        outputs=[outp],
        value_info=[W],
    )

    model = oh.make_model(graph, producer_name="dws_cnv-model")
    model = ModelWrapper(model)
    model.set_tensor_datatype("inp", idt)
    model.set_tensor_datatype("outp", odt)
    model.set_tensor_datatype("W", wdt)
    w_tensor = gen_finn_dt_tensor(wdt, [ofm_ch, 1, k_H, k_W])
    model.set_initializer("W", w_tensor)
        
    model = model.transform(InferShapes())

    ###
    model.save("/tmp/conv_node_test.onnx")
    showInNetron("/tmp/conv_node_test.onnx")
    
    input_tensor = gen_finn_dt_tensor(idt, [1, ifm_ch, ifm_dim_H, ifm_dim_W])
    input_dict = {"inp": input_tensor}
    output_dict = oxe.execute_onnx(model, input_dict)
    expected = output_dict["outp"]

    model = model.transform(LowerConvsToMatMul())
    output_dict = oxe.execute_onnx(model, input_dict)
    produced = output_dict["outp"]
    assert (produced == expected).all()

    # check if created nodes have attributes that indicate depthwise conv
    assert model.get_tensor_sparsity("W") is not None
    im2col_node = getCustomOp(model.graph.node[1])
    assert im2col_node.get_nodeattr("depthwise") == 1

In [27]:
idt = DataType.INT2
k_H = 4
k_W = 2
ifm_dim_H = 4
ifm_dim_W = 2
ifm_ch = 2
stride = 1
padding = [0,1,0,1]

test_depthwise_conv_lowering(idt, k_H, k_W, ifm_dim_H, ifm_dim_W, ifm_ch, stride, padding)


Stopping http://0.0.0.0:8081
Serving '/tmp/conv_node_test.onnx' at http://0.0.0.0:8081
4:1 	 2:3
4:1 	 2:3


In [ ]:
def test_conv_lowering_conv_1x1():

    np.random.seed(0)

    in_feature_dim_H = 7
    in_feature_dim_W = 7
    in_chn = 3
    kernel_size = 1
    out_feature_dim_H = in_feature_dim_H
    out_feature_dim_W = in_feature_dim_W

    input_shape = [1, in_chn, in_feature_dim_H, in_feature_dim_W]
    output_shape = [1, in_chn, out_feature_dim_H, out_feature_dim_W]

    conv_param_shape = [in_chn, in_chn, kernel_size, kernel_size]

    conv_config = {}
    conv_config["dilations"] = [1, 1]
    conv_config["group"] = 1
    conv_config["kernel_shape"] = [kernel_size, kernel_size]
    conv_config["pads"] = [0, 0, 0, 0]
    conv_config["strides"] = [1, 1]

    top_in = oh.make_tensor_value_info("top_in", TensorProto.FLOAT, input_shape)
    top_out = oh.make_tensor_value_info("top_out", TensorProto.FLOAT, output_shape)

    value_info = [oh.make_tensor_value_info("p1", TensorProto.FLOAT, conv_param_shape)]

    modelproto = oh.make_model(
        oh.make_graph(
            name="test",
            inputs=[top_in],
            outputs=[top_out],
            value_info=value_info,
            nodes=[oh.make_node("Conv", ["top_in", "p1"], ["top_out"], **conv_config)],
        )
    )
    model = ModelWrapper(modelproto)
    model = model.transform(InferShapes())
    model.set_initializer("p1", np.random.rand(*conv_param_shape).astype(np.float32))

    new_model = model.transform(LowerConvsToMatMul())
    inp_dict = {"top_in": np.random.rand(*input_shape).astype(np.float32)}

    assert oxe.compare_execution(model, new_model, inp_dict)
    assert new_model.graph.node[0].op_type == "Transpose"
    assert new_model.graph.node[1].op_type == "MatMul"
    assert new_model.graph.node[2].op_type == "Transpose"
    assert len(new_model.graph.node) == 3


In [ ]:
test_conv_lowering_conv_1x1()

In [ ]:
def test_regular_conv_lowering(idt, k_H, k_W, ifm_dim_H, ifm_dim_W, ifm_ch, stride, padding):

    if k_H > ifm_dim_H:
        pytest.skip("Kernel height must be smaller than image height")
    if k_W > ifm_dim_W:
        pytest.skip("Kernel width must be smaller than image height")
    # Ensure the right padding parameters are set
    if ifm_dim_H == 1:
        padding[0] = 0
        padding[2] = 0
    if ifm_dim_W == 1:
        padding[1] = 0
        padding[3] = 0

    wdt = idt
    odt = DataType.INT32
    ofm_ch = ifm_ch
    pad_W = padding[0] + padding[2]
    pad_H = padding[1] + padding[3]
    ofm_dim_H = compute_conv_output_dim_2D_padding(ifm_dim_H, k_H, stride, pad_H)
    ofm_dim_W = compute_conv_output_dim_2D_padding(ifm_dim_W, k_W, stride, pad_W)

    # set up onnx model
    inp = oh.make_tensor_value_info(
        "inp", TensorProto.FLOAT, [1, ifm_ch, ifm_dim_H, ifm_dim_W]
    )
    outp = oh.make_tensor_value_info(
        "outp", TensorProto.FLOAT, [1, ofm_ch, ofm_dim_H, ofm_dim_W]
    )

    W = oh.make_tensor_value_info("W", TensorProto.FLOAT, [ofm_ch, ifm_ch, k_H, k_W])

    dw_cnv = oh.make_node(
        "Conv",
        inputs=["inp", "W"],
        outputs=["outp"],
        kernel_shape=[k_H, k_W],
        pads=padding,
        strides=[stride, stride],
        group=1,
    )
    graph = oh.make_graph(
        nodes=[dw_cnv],
        name="dw_cnv_graph",
        inputs=[inp],
        outputs=[outp],
        value_info=[W],
    )

    model = oh.make_model(graph, producer_name="dws_cnv-model")
    model = ModelWrapper(model)
    model.set_tensor_datatype("inp", idt)
    model.set_tensor_datatype("outp", odt)
    model.set_tensor_datatype("W", wdt)
    w_tensor = gen_finn_dt_tensor(wdt, [ofm_ch, ifm_ch, k_H, k_W])
    model.set_initializer("W", w_tensor)
    model = model.transform(InferShapes())

    input_tensor = gen_finn_dt_tensor(idt, [1, ifm_ch, ifm_dim_H, ifm_dim_W])
    input_dict = {"inp": input_tensor}
    output_dict = oxe.execute_onnx(model, input_dict)
    expected = output_dict["outp"]

    model = model.transform(LowerConvsToMatMul())
    output_dict = oxe.execute_onnx(model, input_dict)
    produced = output_dict["outp"]
    assert (produced == expected).all()

In [ ]:
idt = DataType.INT2
k_H = 4
k_W = 1
ifm_dim_H = 4
ifm_dim_W = 1
ifm_ch = 2
stride = 1
padding = [1,0,1,0]

test_depthwise_conv_lowering(idt, k_H, k_W, ifm_dim_H, ifm_dim_W, ifm_ch, stride, padding)

In [16]:
def _auto_pad_to_explicit_padding(autopad_str, idim_H, idim_W, k_H, k_W, stride, n_dims):
    pad_total_H = (stride - 1) * idim_H - stride + k_H
    pad_total_W = (stride - 1) * idim_W - stride + k_W
    pad_half_small_H = int((pad_total_H / 2))
    pad_half_small_W = int((pad_total_W / 2))
    pad_half_large_H = pad_total_H - pad_half_small_H
    pad_half_large_W = pad_total_W - pad_half_small_W
    if autopad_str == "VALID":
        return [0 for i in range(2 * n_dims)]
    elif autopad_str == "SAME_UPPER":
        return [pad_half_small_H, pad_half_small_W, pad_half_large_H, pad_half_large_W]
    elif autopad_str == "SAME_LOWER":
        return [pad_half_large_H, pad_half_large_W, pad_half_small_H, pad_half_small_W]
    else:
        raise Exception("Unsupported auto_pad: " + autopad_str)
        
a = _auto_pad_to_explicit_padding("SAME_LOWER", 5, 5, 2, 2, 1, 2)        
print(a)

[1, 1, 0, 0]


# Extracting relevant conv node attributes

To do: find out how to map conv node input to what is expected

In [ ]:
import numpy as np
from finn.util.visualization import showInNetron
import onnx
from finn.core.modelwrapper import ModelWrapper
from finn.util.basic import *
from finn.transformation.general import GiveReadableTensorNames, GiveUniqueNodeNames, RemoveStaticGraphInputs
from finn.transformation.infer_shapes import InferShapes
from finn.transformation.infer_datatypes import InferDataTypes
from finn.transformation.fold_constants import FoldConstants


In [ ]:
file_name = "/tmp/quartznet.onnx"
#model = ModelWrapper(file_name)
#model = model.transform(InferShapes())
#model = model.transform(FoldConstants())
#model = model.transform(GiveUniqueNodeNames())
#model = model.transform(GiveReadableTensorNames())
#model = model.transform(InferDataTypes())
#model = model.transform(RemoveStaticGraphInputs())
#model.save("/tmp/quartznet_cleaned.onnx")
file_name = "/tmp/quartznet_cleaned.onnx"


In [ ]:
#idt, k_H, k_W, ifm_dim_H, ifm_dim_W, ifm_ch, stride, padding)
model = ModelWrapper(file_name)
conv_node_count=0
idt = []
k_W = []
ifm_dim_W = []
ifm_ch = []
stride = []
padding = []
group = []
for n in model.graph.node:
    if(n.op_type=="Conv"):
        conv_node_count = conv_node_count+1
        input_name = n.input[0]
        input_datatype = model.get_tensor_datatype(input_name)
        idt.append(input_datatype)
        
        weight_name = n.input[1]
        weight_shape = model.get_tensor_shape(weight_name)
        k_W.append(weight_shape[2])

        input_shape = model.get_tensor_shape(input_name)
        ifm_dim_W.append(input_shape[2])

        ifm_ch.append(input_shape[1])
        
        strds = get_by_name(n.attribute, "strides", "name").ints[0]
        stride.append(strds)
        
        pads = [0, 0, get_by_name(n.attribute, "pads", "name").ints[0], get_by_name(n.attribute, "pads", "name").ints[1]] # Note: I assumed dim(H)=1
        padding.append(pads)
        
        grp = get_by_name(n.attribute, "group", "name").i
        group.append(grp)
        
ifm_dim_H = [1]*conv_node_count        
k_H = [1]*conv_node_count

#print("{}\n{}".format(len(padding),padding))

In [ ]:
for i in range(1):
    print(group[i])
    test_depthwise_conv_lowering(DataType.INT8, k_H[i], k_W[i], ifm_dim_H[i], ifm_dim_W[i], ifm_ch[i], stride[i], padding[i], group[i])
    print("Test {} passed" .format(i))
    
#idt = DataType.INT2
#k_H = k[0][0]
#k_W = k[0][1]
#ifm_dim = [[4,4]]
#ifm_dim_H = ifm_dim[0][0]
#ifm_dim_W = ifm_dim[0][1]
#ifm_ch = 2
#stride = 1
#padding = [0,0,0,0]    